In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np 
import pandas as pd 
import os
import jieba
import jieba.analyse
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split  

def cuttrain_data(df):
  cuttitle = []
  for d in df['title'].astype(str):
    cut = jieba.analyse.extract_tags(d,topK=2)
    cuttitle.append(cut)
  print("in cuttitle")
  # for d in cuttitle:
  #   print(d)
  cutcontext = []
  for d in df['context']:
    cut = jieba.analyse.extract_tags(d,topK=2)
    cutcontext.append(cut)
  # print("in context")
  # for d in cutcontext:
  #   print(d)
  push = []
  for d in df['push']:
    push.append(d)
  down = []
  for d in df['down']:
    down.append(d)

  data = {'title':cuttitle ,'context':cutcontext ,'push':push ,'down':down}
  df1 = pd.DataFrame(data)

  df1['context'] = df1['context'].apply(lambda x : str(x)[1:-2].replace("'","").replace(",",""))
  # print("df1['context'] = ")
  # print(df1['context'])
  df1['title'] = df1['title'].apply(lambda x : str(x)[1:-2].replace("'","").replace(",",""))
  # print("df1['title'] = ")
  # print(df1['title'])
  df1['push_down'] = df1['title'].astype(str)+' '+df1['context'].astype(str)+' '+df1['push'].astype(str)+' '+df1['down'].astype(str)

  return df1

def cuttest_data(df):
  cuttitle = []
  for d in df['title'].astype(str):
    cut = jieba.analyse.extract_tags(d,topK=2)
    cuttitle.append(cut)
  cutcontext = []
  for d in df['context']:
    cut = jieba.analyse.extract_tags(d,topK=10)
    cutcontext.append(cut)

  data = {'title':cuttitle ,'context':cutcontext}
  df1 = pd.DataFrame(data)

  df1['context'] = df1['context'].apply(lambda x : str(x)[1:-2].replace("'","").replace(",",""))
  df1['title'] = df1['title'].apply(lambda x : str(x)[1:-2].replace("'","").replace(",",""))
  df1['key'] = df1['title'].astype(str)+' '+df1['context'].astype(str)
  return df1

def count(df,df2):
  vectorizer = CountVectorizer()
  X = vectorizer.fit_transform(df['push_down'])
  X2 = vectorizer.transform(df2['key'])
  tfidf = TfidfTransformer() 
  tf = tfidf.fit_transform(X)
  tf2 = tfidf.transform(X2)
  return tf ,tf2

def knn_push(tf_train,train,tf_test):
  X_train, X_test, y_train, y_test = train_test_split(tf_train, train['push'].astype(str), test_size=0.2, random_state=1)
  clf=KNeighborsClassifier(n_neighbors=70, p=2)
  clf.fit(X_train,y_train)
  y_pred = clf.predict(X_test)
  ans_push = y_pred
  test_p = clf.predict(tf_test)
  # print(len(test_p))
  return test_p

def knn_down(tf_train,train,tf_test):
  X_train, X_test, y_train, y_test = train_test_split(tf_train, train['down'].astype(str), test_size=0.2, random_state=1)
  clf=KNeighborsClassifier(n_neighbors=70, p=2)
  clf.fit(X_train,y_train)
  y_pred = clf.predict(X_test)
  ans_push = y_pred 
  test_d = clf.predict(tf_test)
  print(len(test_d))
  return test_d



test_index=[]
def save(test_index,test_p,test_d,test):
  for i in range(len(test)):
    test_index.append(i)
  data = {'index': test_index, 'push': test_p, 'down': test_d}
  export = pd.DataFrame(data)
  export.to_csv('projectfinal.csv' ,index = False)



if __name__ == '__main__':
  train_data = pd.read_csv("/content/drive/MyDrive/NatureLanguageProgram/NLP_final_project/train_data.csv")
  train_label = pd.read_csv("/content/drive/MyDrive/NatureLanguageProgram/NLP_final_project/train_label.csv")
  test_data = pd.read_csv("/content/drive/MyDrive/NatureLanguageProgram/NLP_final_project/test_data.csv")
  files = glob('/content/drive/MyDrive/NatureLanguageProgram/NLP_final_project/train*.csv')
  df_list = [pd.read_csv(file) for file in files]  
  total_train_data = pd.merge(df_list[0], df_list[1], on='index')

  the_training_data = total_train_data[['title' ,'context' ,'push' ,'down']]
  the_test_data = test_data[['title' ,'context']]

  train = cuttrain_data(the_training_data)

  test = cuttest_data(the_test_data) #31849
  # print("len(test)")
  # print(len(test))

  tf_train , tf_test = count(train,test)
  # tf_test = count_test(test,vector,tfidf)
  # print("len(tf_train)")
  # print(tf_train.shape)


  test_p = knn_push(tf_push,train,tf_test)

  test_d = knn_down(tf_down,train,tf_test)

  save(test_index,test_p,test_d,test)

  my_answer= pd.read_csv("projectfinal.csv")
  print("ans")
  my_answer



in cuttitle
